# CNN_3channel_Fit_VGG19_FineTune

Abstract:
- REsize image to fit VGG19
- CNN, small net

Result:
- Kaggle score: 

References:
- https://www.kaggle.com/ivalmian/simple-svd-xgboost-baseline-lb-35
- https://www.kaggle.com/arieltci/a-keras-prototype-0-21174-on-pl
- https://www.kaggle.com/henokanh/cnn-batchnormalization-0-1646

## 1. Preprocess

### Import pkgs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, accuracy_score
from IPython.display import display

%matplotlib inline

In [2]:
import os
import time
import zipfile
import lzma
import pickle
from PIL import Image
from shutil import copy2
import gc

### Run name

In [3]:
project_name = 'SC_Iceberg_Classifier'
step_name = 'CNN_3channel_Fit_VGG19_FineTune'
date_str = time.strftime("%Y%m%d", time.localtime())
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: SC_Iceberg_Classifier_CNN_3channel_Fit_VGG19_FineTune_20180107_132211


### Basic folders

In [4]:
cwd = os.getcwd()
input_path = os.path.join(cwd, 'input')
log_path = os.path.join(cwd, 'log')
model_path = os.path.join(cwd, 'model')
output_path = os.path.join(cwd, 'output')
print('input_path: ' + input_path)
print('log_path: ' + log_path)
print('model_path: ' + model_path)
print('output_path: ' + output_path)

input_path: /data1/Kaggle/statoil-iceberg-classifier-challenge/input
log_path: /data1/Kaggle/statoil-iceberg-classifier-challenge/log
model_path: /data1/Kaggle/statoil-iceberg-classifier-challenge/model
output_path: /data1/Kaggle/statoil-iceberg-classifier-challenge/output


### Basic functions

In [5]:
import sys
def describe(arr):
    print(arr.shape, arr.min(), arr.max(), sys.getsizeof(arr))

### Load data

In [6]:
sample_submission_path = os.path.join(input_path, 'sample_submission.csv')
sample_submission = pd.read_csv(sample_submission_path)
print(sample_submission.shape)
sample_submission.head(2)

(8424, 2)


,id,is_iceberg
0,5941774d,0.5
1,4023181e,0.5


In [7]:
is_iceberg_path = os.path.join(input_path, 'is_iceberg.p')
y_data = pickle.load(open(is_iceberg_path, mode='rb'))

describe(y_data)

(1604,) 0 1 25688


In [8]:
%%time
#Load orignal data
inc_angle_data_path = os.path.join(input_path, 'inc_angle_data.p')
inc_angle_test_path = os.path.join(input_path, 'inc_angle_test.p')

inc_angle_data = pickle.load(open(inc_angle_data_path, mode='rb'))
inc_angle_test = pickle.load(open(inc_angle_test_path, mode='rb'))

describe(inc_angle_data)
describe(inc_angle_test)

(1604,) 0.0 45.9375 25688
(8424,) 23.0805 50.66178518 134808
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.18 ms


In [9]:
%%time
#Load orignal data
# band1_data_path = os.path.join(input_path, 'band1_data_gray200.p')
# band2_data_path = os.path.join(input_path, 'band2_data_gray200.p')
# band1_test_path = os.path.join(input_path, 'band1_test_gray200.p')
# band2_test_path = os.path.join(input_path, 'band2_test_gray200.p')

band1_data_path = os.path.join(input_path, 'band1_data_gray.p')
band2_data_path = os.path.join(input_path, 'band2_data_gray.p')
band1_test_path = os.path.join(input_path, 'band1_test_gray.p')
band2_test_path = os.path.join(input_path, 'band2_test_gray.p')

band1_data = pickle.load(open(band1_data_path, mode='rb'))
band2_data = pickle.load(open(band2_data_path, mode='rb'))
band1_test = pickle.load(open(band1_test_path, mode='rb'))
band2_test = pickle.load(open(band2_test_path, mode='rb'))

band_avg_data = (band1_data + band2_data)/2
band_avg_test = (band1_test + band2_test)/2


describe(band1_data)
describe(band2_data)
describe(band1_test)
describe(band2_test)

describe(band_avg_data)
describe(band_avg_test)

(1604, 75, 75) 0.0 1.0 128
(1604, 75, 75) 0.0 1.0 128
(8424, 75, 75) 0.0 1.0 128
(8424, 75, 75) 0.0 1.0 128
(1604, 75, 75) 0.0114754206838 1.0 72180128
(8424, 75, 75) 0.0 1.0 379080128
CPU times: user 624 ms, sys: 1.12 s, total: 1.74 s
Wall time: 1.74 s


In [10]:
%%time
x_data = np.concatenate([band1_data[:, :, :, np.newaxis],
                         band2_data[:, :, :, np.newaxis],
                         band_avg_data[:, :, :, np.newaxis]], axis=-1)
describe(x_data)
del band1_data
del band2_data
del band_avg_data
gc.collect()
x_test = np.concatenate([band1_test[:, :, :, np.newaxis],
                         band2_test[:, :, :, np.newaxis],
                         band_avg_test[:, :, :, np.newaxis]], axis=-1)
describe(x_test)
del band1_test
del band2_test
del band_avg_test
gc.collect()

(1604, 75, 75, 3) 0.0 1.0 216540144
(8424, 75, 75, 3) 0.0 1.0 1137240144
CPU times: user 852 ms, sys: 524 ms, total: 1.38 s
Wall time: 1.37 s


In [11]:
%%time
x_train, x_val, inc_angle_train, inc_angle_val, y_train, y_val = train_test_split(x_data, inc_angle_data, y_data, test_size=0.15, random_state=31)
print(x_train.shape)
print(x_val.shape)
print(inc_angle_train.shape)
print(inc_angle_val.shape)
print(y_train.shape)
print(y_val.shape)

(1363, 75, 75, 3)
(241, 75, 75, 3)
(1363,)
(241,)
(1363,)
(241,)
CPU times: user 40 ms, sys: 80 ms, total: 120 ms
Wall time: 119 ms


## 2. Build model

In [12]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


In [13]:
def build_model():
    base_model = VGG19(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [14]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [15]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
# saveModel(model, 'saveModel_test')

In [16]:
def get_lr(x):
    lr = round(1e-4 * 0.995 ** x, 6)
    if lr < 5e-5:
        lr = 5e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

log_dir = os.path.join(log_path, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:/data1/Kaggle/statoil-iceberg-classifier-challenge/log/SC_Iceberg_Classifier_CNN_3channel_Fit_VGG19_FineTune_20180107_132211


In [ ]:
datagen = ImageDataGenerator(
    zoom_range = 0.2,
    rotation_range=20,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:
%%time
batch_size = 64
# steps_per_epoch = 1 * len(x_train) / batch_size
hist = model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=True),
    steps_per_epoch=100,
    epochs=100, #1 for ETA, 0 for silent
    verbose=1,
    max_queue_size=128,
    callbacks=[annealer],
    workers=32,
    validation_data=(x_val, y_val)
)

0.0001  Epoch 1/100
100/100 [==============================] - 29s 285ms/step - loss: 0.6162 - acc: 0.6652 - val_loss: 0.5698 - val_acc: 0.6598
0.0001  Epoch 2/100
100/100 [==============================] - 26s 256ms/step - loss: 0.5619 - acc: 0.7078 - val_loss: 0.5334 - val_acc: 0.7012
9.9e-05  Epoch 3/100
100/100 [==============================] - 24s 244ms/step - loss: 0.5303 - acc: 0.7159 - val_loss: 0.5059 - val_acc: 0.7220
9.9e-05  Epoch 4/100
100/100 [==============================] - 25s 254ms/step - loss: 0.5256 - acc: 0.7270 - val_loss: 0.4864 - val_acc: 0.7137
9.8e-05  Epoch 5/100
100/100 [==============================] - 25s 245ms/step - loss: 0.5007 - acc: 0.7452 - val_loss: 0.4716 - val_acc: 0.7220
9.8e-05  Epoch 6/100
100/100 [==============================] - 25s 254ms/step - loss: 0.4935 - acc: 0.7472 - val_loss: 0.4637 - val_acc: 0.7137
9.7e-05  Epoch 7/100
100/100 [==============================] - 24s 243ms/step - loss: 0.4874 - acc: 0.7496 - val_loss: 0.4638 - val

100/100 [==============================] - 26s 256ms/step - loss: 0.4098 - acc: 0.7961 - val_loss: 0.3758 - val_acc: 0.8050
7.5e-05  Epoch 58/100
100/100 [==============================] - 24s 244ms/step - loss: 0.4174 - acc: 0.7896 - val_loss: 0.3845 - val_acc: 0.7925
7.5e-05  Epoch 59/100
100/100 [==============================] - 26s 256ms/step - loss: 0.4058 - acc: 0.7998 - val_loss: 0.3853 - val_acc: 0.8008
7.4e-05  Epoch 60/100
100/100 [==============================] - 24s 243ms/step - loss: 0.4066 - acc: 0.7961 - val_loss: 0.3787 - val_acc: 0.8174
7.4e-05  Epoch 61/100
100/100 [==============================] - 25s 255ms/step - loss: 0.4102 - acc: 0.7975 - val_loss: 0.3762 - val_acc: 0.8216
7.4e-05  Epoch 62/100
100/100 [==============================] - 24s 244ms/step - loss: 0.4075 - acc: 0.8014 - val_loss: 0.4227 - val_acc: 0.7635
7.3e-05  Epoch 63/100
100/100 [==============================] - 26s 259ms/step - loss: 0.4214 - acc: 0.7892 - val_loss: 0.4057 - val_acc: 0.7635


In [ ]:
# %%time
# hist = model.fit(x_train, y_train, 
#                  batch_size = 2, 
#                  verbose= 1,
#                  epochs = 10, #1 for ETA, 0 for silent
#                  validation_data=(x_val, y_val))

In [ ]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
val_prob1 = model.predict(x_val)

# print('Val log_loss: {}'.format(log_loss(y_val, val_prob1)))
val_prob1_limit = np.clip(val_prob1, 0.00005, 0.99995)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

val_prob1_limit = np.clip(val_prob1_limit, 0.05, 0.95)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

In [ ]:
final_acc_str = '{0:0>4}'.format(int(loss*10000))
run_name_acc = project_name + '_' + step_name + '_' + time_str + '_' + final_acc_str
print(run_name_acc)

In [ ]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_path, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

In [ ]:
plt.plot(histories['loss'], color='b')
plt.plot(histories['val_loss'], color='r')
plt.show()
plt.plot(histories['acc'], color='b')
plt.plot(histories['val_acc'], color='r')
plt.show()

In [ ]:
# for layer in model.layers[:11]:
#    layer.trainable = False
# for layer in model.layers[11:]:
#    layer.trainable = True

# from keras.optimizers import SGD
# # model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
# model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# %%time
# hist = model.fit(x_train, y_train, 
#                  batch_size = 2, 
#                  verbose= 1,
#                  epochs = 10, #1 for ETA, 0 for silent
#                  validation_data=(x_val, y_val))

In [ ]:
saveModel(model, run_name_acc)

## 3. Predict

In [ ]:
if not os.path.exists(output_path):
    os.mkdir(output_path)
pred_file = os.path.join(output_path, run_name_acc + '.csv')
print(pred_file)

In [ ]:
test_prob = model.predict(x_test)
print(test_prob.shape)
print(test_prob[0:2])
test_prob = np.clip(test_prob, 0.05, 0.95)
print(test_prob.shape)
print(test_prob[0:2])

In [ ]:
sample_submission['is_iceberg'] = test_prob
print(sample_submission[0:2])
print(sample_submission.shape)
sample_submission.to_csv(pred_file, index=False)

In [ ]:
t1 = time.time()
print('time cost: %.2f s' % (t1-t0))
print('Done!')

In [ ]:
print(run_name_acc)